# Livrable projet 4 :Nettoyage des données

In [1]:
# Import des librairies
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import re

# Import des données des bilans alimentaires
cus = pd.read_csv("data/customers.csv")
pro = pd.read_csv("data/products.csv")
tra = pd.read_csv("data/transactions.csv",parse_dates=[1])



Nous avons ci-dessous les fonctions qui vont nous permettre de nettoyer et formater les différentes données

In [2]:
def birth_verif(value): 
    """Vérification que la date de naissance est bien une date possible
    """
    date = datetime.datetime.now()

    if value > 1900 and value < date.year:
        return value
    return np.NaN


def sex_verif(value):
    """Vérification que le sexe entré est un sexe qui existe, nous avons pour ça la liste VALID_SEX
    """
    VALID_SEX = ['f','m']

    if value not in VALID_SEX:
        return np.NaN
    return value


def clientId_verif(value):
    """Vérification que tous les identifiants clients on la même architecture
    """
    found = re.search('c_\d*', value)
    if found is None:
        return np.NaN
    return value


def idProd_verif(value):
    """Fonction qui verifie que l'identifiant de tous les produits suis la même logique
    """
    found = re.search('^[0-2]_\d+', value)
    if found is None:
        return np.NaN
    return value
    
    
def sessionId_verif(value):
    """Fonction qui verifie l'identifiant des sessions
    """
    found = re.search('^s_\d+', value)
    if value is None:
        return np.NaN
    return value

  
def price_verif(value):
    """Fonction qui verifie qu'un prix est positif
    """
    if value < 0:
        return np.NaN
    return value


def categ_verif(value):
    """Verification que la categorie est une categorie qui existe, la liste des categories existante est donner par VALID_CATEG
    """
    VALID_CATEG = [0,1,2]

    if value not in VALID_CATEG:
        return np.NaN
    return value
    

def date_verif(value):
    """Cette fonction verifie que les dates soit bien toute au même formats, a savoir que des digits
    """
    found = re.search('^\D+', value)
    if found is None:
        return value
    else:
         return np.NaN
        


Nous effectuons ainsi notre formatage

In [3]:
cus['birth'] = cus['birth'].apply(birth_verif)
cus['sex'] = cus['sex'].apply(sex_verif)
cus['client_id'] = cus['client_id'].apply(clientId_verif)


In [4]:
pro['id_prod'] = pro['id_prod'].apply(idProd_verif)
pro['price'] = pro['price'].apply(price_verif)
pro['categ'] = pro['categ'].apply(categ_verif)

#Ici on regarde que tout les identifiant et toute les categorie concorde
pro[pro['id_prod'].astype(str).str[0] != pro['categ'].astype(str).str[0]]


,id_prod,price,categ
731,NaN,NaN,0


Nous n'avons que le produits Nan qui est le produits test

In [5]:
tra['id_prod'] = tra['id_prod'].apply(idProd_verif)
tra['date'] = tra['date'].apply(date_verif)
tra['session_id'] = tra['session_id'].apply(sessionId_verif)
tra['client_id'] = tra['client_id'].apply(clientId_verif)
tra['date'] = pd.to_datetime(tra['date'])

In [6]:
cus.isna().sum()


client_id    2
sex          0
birth        0
dtype: int64

In [7]:
pro.isna().sum()


id_prod    1
price      1
categ      0
dtype: int64

In [8]:
tra.isna().sum()


id_prod       200
date          200
session_id      0
client_id     200
dtype: int64

In [9]:
tra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     336816 non-null  object        
 1   date        336816 non-null  datetime64[ns]
 2   session_id  337016 non-null  object        
 3   client_id   336816 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 10.3+ MB


Nous pouvons remarquer que nous avons plusieurs valeurs manquantes suite à notre formatage


- *customer* -> 8623 entrées -> 2 corrompus soit 0.02% des entrées
- *produits* -> 3287 entrées -> 1 corrompu soit 0.03% des entrées
- *transaction* -> 337 016 entrées -> 200 corrompus soit 0.05% des entrées


Notre échantillon, même sans ces pourcentages de pertes, sera toujours représentatif, il n'y aura aucun impact de paniers moyens ou autres car ces "erreur" se trouve sur des articles avec un prix négatif

In [10]:
cus = cus.dropna()
pro = pro.dropna()
tra = tra.dropna()

In [11]:
cus.to_csv("data/customer_formate.csv",encoding="utf-8",index =False)

In [12]:
pro.to_csv("data/product_formate.csv",encoding="utf-8",index =False)

In [13]:
tra.to_csv("data/transaction_formate.csv",encoding="utf-8",index =False)